In [ ]:
import time
start_time = time.time()
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import xlsxwriter

#autoarima
from pmdarima.arima import auto_arima
import statsmodels.api as sm
import arch
from arch import arch_model
import math

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.layers import LSTM

ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def mean_of_highest_3(x):
    return x.nlargest(3).mean()

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def convert_seconds_to_hours_minutes(seconds):
    # Calculate the number of hours
    hours = seconds // 3600

    # Calculate the number of minutes
    minutes = (seconds % 3600) // 60

    print ("Time elapsed - ", hours,"hours and ", minutes,"minutes")

In [ ]:
# dataframe
df=pd.read_excel("till_august_AB.xlsx", parse_dates=['date'], index_col='date')

In [ ]:
df

,600153,600234,600235,600307,600308,600310,600311,600315,600319,600457,...,610055,610196,610215,610236,610323,610344,610381,610384,610406,610425
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,14.0,16.0,25.0,9929,8886,8.0,12015,2496,19343,8723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-02-01,14.0,14.0,28.0,9532,7253,7.0,14583,2393,19456,10127,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-01,26.0,14.0,19.0,6754,8929,5.0,11513,2297,18110,11191,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-04-01,14.0,12.0,26.0,4298,7262,2.0,6338,2268,13735,8606,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-05-01,14.0,18.0,26.0,6884,6648,7.0,9230,3249,16577,10943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-01,29.0,14.0,18.0,4156,8711,7.0,9927,2889,23759,8585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-01,18.0,17.0,28.0,12494,6671,4.0,9230,1266,16498,9854,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-08-01,18.0,11.0,23.0,5877,7658,6.0,12126,2049,17164,10015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-01,12.0,6.0,23.0,17108,7321,2.0,16526,4182,17317,11013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 56 entries, 2019-01-01 to 2023-08-01
Columns: 916 entries, 600153 to 610425
dtypes: float64(495), int64(421)
memory usage: 401.2 KB


In [ ]:
final_df = pd.DataFrame()
final_df_selected = pd.DataFrame()

removed_SKU=[]

df7=df.copy()


for col in df7:
#for col in df.columns:

    item_j = pd.DataFrame({col: df7[col]})
    item_j=item_j.iloc[-12:, 0:]
    item_j=item_j.clip(lower=0)
    item_j= item_j.replace(0, np.nan)
    item_j=item_j.dropna()

    if len(item_j) < 6 :
        print( 'Removed SKU ID :',col)
        df7 = df7.drop(col, axis=1)
        removed_SKU.append(col)


for col in df7:
#for col in df.columns:

    item_j = pd.DataFrame({col: df7[col]})
    item_j=item_j.iloc[-36:-12, 0:]
    item_j=item_j.clip(lower=0)
    item_j = item_j.replace(0, np.nan)
    item_j=item_j.dropna()

    if len(item_j) < 18 :
        print( 'Removed SKU ID :',col)
        df7 = df7.drop(col, axis=1)
        removed_SKU.append(col)


print('removed SKU count',len(removed_SKU))
column_names = ['Skipped']
df_removed = pd.DataFrame(removed_SKU, columns=column_names)

#print('removed SKU from the list',df_removed)



print('Start')


count = 0
for col in df7.iloc[:, 0:5]:
#for col in df.columns:
    new_df = pd.DataFrame({col: df7[col]})
    item_i=new_df.copy()
    item_q=new_df.copy()

    count+=1
    print(count,'SKU',col)

    #replace negatives by 0
    new_df=new_df.clip(lower=0)


    #1st non null
    item_q = item_q.replace(0, np.nan)
    first_non_null_index = item_q.iloc[:, 0].first_valid_index()
    new_df = new_df.loc[first_non_null_index:]


    #missing value imputation
    rolling_avg = new_df.rolling(window=3, min_periods=1, center=True).mean()
    new_df[new_df.isna()] = rolling_avg[new_df.isna()]

    #exp smoothed
    span = 6
    ema_df = new_df.ewm(span=span, adjust=False).mean()


    #outlier detection

    #rolling_avg = new_df.rolling(window=3, min_periods=1, center=True).mean()
    abs_deviation = pd.Series(abs(new_df - ema_df).values.flatten(),index=new_df.index)
    mad = abs_deviation.median()
    threshold = mad*2.5
    outlier_mask = abs_deviation > threshold
    new_df[outlier_mask] = ema_df[outlier_mask]


    new_df = new_df.fillna(0)

    item_i_smooth=new_df.copy()


    print('length',len(item_i_smooth))

    for i in range(1, 4):
        new_df[f'shifted_t{i}'] = new_df[col].shift(i)
    #print(new_df)

    new_df = new_df.dropna()
    #df=new_df
    k_list = []
    lr_a = []
    xg_a = []
    svr_a=[]
    rf_a =[]
    start = 0
    end = len(new_df)-12
    # loop through the rows of the dataframe and fit models on subsets of data
    while end <= len(new_df):
        dfn = new_df[start:end]
        y = dfn.drop(['shifted_t1','shifted_t2','shifted_t3'], axis=1)
        X = dfn[['shifted_t1','shifted_t2','shifted_t3']]
        #print(y.columns[0])
        end += 1
        # split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        # create and fit the linear regression model
        predictions_list = []
        models = [LinearRegression(),XGBRegressor(),SVR(),RandomForestRegressor()]
        for model in models:
            #print(model)
            model.fit(X_train, y_train)
            # get the last row of the data frame to start the prediction loop
            X_next = X.iloc[-1, :len(dfn)].values.reshape(1, -1)  # reshape to a 2D array for compatibility with model.predict()
        # use a loop to predict the next 6 values
            for i in range(6):
                y_pred_next = model.predict(X_next)
                #print('Predicted value for time step',+ i + 1, ':', y_pred_next[0])
                X_next = np.append(X_next[:, 1:], y_pred_next).reshape(1, -1)
                y_pred_next=y_pred_next.tolist()
                predictions_list.append(y_pred_next)
                #print()
            #print("")
        k_list.append(predictions_list)
        lr_a.append(predictions_list[5])
        del lr_a[12::13]
        df_lra = pd.DataFrame([i[0] for i in lr_a])
        xg_a.append(predictions_list[11])
        del xg_a[12::13]
        df_xga = pd.DataFrame(xg_a)
        svr_a.append(predictions_list[17])
        del svr_a[12::13]
        df_svra =pd.DataFrame (svr_a)
        rf_a.append(predictions_list[23])
        del rf_a [12::13]
        df_rfa = pd.DataFrame(rf_a)
        k_list[12::13]
        P_P=[l[0] for i in k_list[12::13] for l in i]
        lr_P=P_P[:6]
        df_lrP=pd.DataFrame([i[0] for i in lr_P])
        xg_P=P_P[6:12]
        df_xgP=pd.DataFrame(xg_P)
        svr_p = P_P[12:18]
        df_svrP =pd.DataFrame(svr_p)
        rf_p = P_P[18:24]
        df_rfP = pd.DataFrame(rf_p)




    #arima + garch model training
    data = item_i_smooth
    forecast_horizon = 6



    predictions_gar = []
    predictions_ar = []

    #for i in range(window_size, len(data) - forecast_horizon + 1):
    #for i in range(window_size, len(data) + 1):
    for i in range(len(data)-18, len(data) + 1):
        # Select the data points for the current window
        window_data = data.iloc[0:i]

        # Train an ARIMA model on the window data
        arima_model =  auto_arima(window_data,start_p=0, d=1, q=0,
                              max_p=6, max_d=2, max_q=6, start_P=0,
                              D=0, start_Q=0, max_P=2, max_D=2,
                              max_Q=2, m=12, seasonal=True,
                              error_action='warn',trace = False,
                              supress_warnings=True,stepwise = True,
                              random_state=20,n_fits = 50 )



        # ARIMA
        forecast_arima = arima_model.predict(n_periods = forecast_horizon)


        m6_ahead_ar= forecast_arima.tail(1)
        predictions_ar.append(m6_ahead_ar)



        # ARIMA+-GARCH
        residuals = arima_model.resid()
        garch_model = arch_model(residuals, vol='GARCH', p=1, o=0, q=1)
        garch_results = garch_model.fit(disp='off')
        forecast_horizon_garch = garch_results.forecast(horizon=forecast_horizon)
        forecast_volatility = np.sqrt(forecast_horizon_garch.variance.values[-1,:])
        #vol=np.sign(residuals[-1])*forecast_volatility
        vol=forecast_volatility


        forecast_gar = forecast_arima + vol
        predictions_gar.append(forecast_gar)

        m6_ahead_gar= forecast_gar.tail(1)
        predictions_gar.append(m6_ahead_gar)



    #forecast_arima and forecast_gar holds rolling forecast for last 6 months
    #print('arima last 6',forecast_arima)

    #All 6 months ahead predictions
    predictions_ar = pd.concat(predictions_ar, axis=0)
    predictions_gar = pd.concat(predictions_gar, axis=0)

    # for MAPE
    act=item_i_smooth.iloc[-12:].copy()

    #past 12 months predictions - arima n garch
    ar=predictions_ar.iloc[1:13]
    arg=predictions_gar.iloc[1:13]


    #past 12 months predictions - highest 3 of 6
    mean_h3 = item_i_smooth.rolling(6).apply(mean_of_highest_3, raw=False)
    mean_h3 = pd.DataFrame(mean_h3)
    mean_h32=mean_h3.iloc[-18:-6,:].copy()
    mean_h33= mean_h32.set_index(act.index)


    if len(ar) < 12:
        ar1=ar.iloc[:,0].tolist()
        for i in range(0,12-len(ar)):
            ar1.insert(0,0)
        ar = pd.DataFrame(ar1, columns=['0'])


    len(arg)
    if len(arg) < 12:
        arg1=arg.iloc[:,0].tolist()
        for i in range(0,12-len(arg)):
            arg1.insert(0,0)
        arg = pd.DataFrame(arg1, columns=['0'])



    #accuracy measure(+-diviation)

    act.index=range(1,13)
    ar.index=range(1,13)
    arg.index=range(1,13)
    mean_h32.index=range(1,13)
    df_lra.index=range(1,13)
    df_xga.index=range(1,13)
    df_rfa.index =range(1,13)
    df_svra.index = range (1,13)


    combined_df = pd.concat([act, ar, arg, mean_h32,df_lra,df_xga,df_svra,df_rfa], axis=1, join='outer', ignore_index=True)
    # where 0 to nan when cal mape

    combined_df = combined_df.replace(0, np.nan)


    mape_ar = mape(combined_df[0], combined_df[1])
    mape_arg = mape(combined_df[0], combined_df[2])
    mape_mean = mape(combined_df[0], combined_df[3])
    mape_lr = mape(combined_df[0], combined_df[4])
    mape_xg = mape(combined_df[0], combined_df[5])
    mape_svr = mape(combined_df[0],combined_df[6])
    mape_rf = mape (combined_df[0],combined_df[7])



    # next 6 months prediction values
    #index
    y1 = forecast_arima
    x1 = forecast_gar

    z1= mean_h33.tail(6)
    z1= z1.set_index(y1.index)

    df_lr=df_lrP.set_index(y1.index)
    df_xg=df_xgP.set_index(y1.index)
    df_svr=df_svrP.set_index(y1.index)
    df_rf = df_rfP.set_index(y1.index)



    #output doc

    new_index = [1, 2, 3, 4, 5, 6, 7]

    df_name= pd.DataFrame({'model':['GARCH', 'ARIMA', '3MA', 'Linear_R' , 'XG','SVR','RandomF']})
    df_name = df_name.set_index([pd.Index(new_index)])

    df_mape=pd.DataFrame({'MAPE':[mape_arg,mape_ar,mape_mean,mape_lr,mape_xg,mape_svr,mape_rf]})
    df_mape = df_mape.set_index([pd.Index(new_index)])

    df_id = pd.DataFrame({'idx': [col,'','','','','','']})
    df_id2 = pd.DataFrame({'idx': [col,col,col,col,col,col,col]})
    df_id = df_id.set_index([pd.Index(new_index)])
    df_id2 = df_id2.set_index([pd.Index(new_index)])

    df_p = pd.concat([x1,y1,z1,df_lr,df_xg,df_svr,df_rf],axis=1)
    df_p = df_p.reset_index(drop=True)
    df_p = df_p.transpose()
    df_p = df_p.set_index([pd.Index(new_index)])

    actual_d=item_i.iloc[-12:].copy()

    df_act = pd.concat([actual_d,actual_d,actual_d,actual_d,actual_d,actual_d,actual_d],axis=1)
    df_act = df_act.reset_index(drop=True)
    df_act = df_act.transpose()
    df_act = df_act.set_index([pd.Index(new_index)])

    df_SKU_j=pd.concat([df_id,df_name,df_mape,df_act,df_p],axis=1)
    df_SKU_j2=pd.concat([df_id2,df_name,df_mape,df_act,df_p],axis=1)

    #df_SKU_j2 is a dataframe similar to df_SKU_j2 with df_id2

    #sheet2


    min_mape_idx = df_SKU_j2['MAPE'].idxmin()  # get index of row with minimum "mape" value
    min_mape_row = df_SKU_j2.loc[min_mape_idx]

    final_df_selected = final_df_selected.append(min_mape_row)


    #print(df_SKU_j)

    final_df = final_df.append(df_SKU_j)


print('finished')
#print(final_df_selected)
print('Time elapsed: ',time.time() - start_time)
convert_seconds_to_hours_minutes(time.time() - start_time)



Removed SKU ID : 600234
Removed SKU ID : 600235
Removed SKU ID : 601181
Removed SKU ID : 602403
Removed SKU ID : 602575
Removed SKU ID : 602815
Removed SKU ID : 604056
Removed SKU ID : 604244
Removed SKU ID : 604505
Removed SKU ID : 604530
Removed SKU ID : 604531
Removed SKU ID : 604550
Removed SKU ID : 604899
Removed SKU ID : 604900
Removed SKU ID : 604904
Removed SKU ID : 604923
Removed SKU ID : 605094
Removed SKU ID : 605238
Removed SKU ID : 605245
Removed SKU ID : 605248
Removed SKU ID : 605262
Removed SKU ID : 605265
Removed SKU ID : 605274
Removed SKU ID : 605286
Removed SKU ID : 605313
Removed SKU ID : 605332
Removed SKU ID : 605429
Removed SKU ID : 605478
Removed SKU ID : 605616
Removed SKU ID : 605714
Removed SKU ID : 605719
Removed SKU ID : 605724
Removed SKU ID : 605784
Removed SKU ID : 605823
Removed SKU ID : 605876
Removed SKU ID : 606097
Removed SKU ID : 606287
Removed SKU ID : 606501
Removed SKU ID : 606606
Removed SKU ID : 606774
Removed SKU ID : 606886
Removed SKU ID :

In [ ]:
final_df.head(10)

,idx,model,MAPE,0,1,2,3,4,5,6,...,8,9,10,11,0,1,2,3,4,5
1,600153,GARCH,100.058283,12.0,9.0,16.0,16.0,18.0,0.0,6.0,...,13.0,20.0,15.0,14.0,18.254167,19.005672,19.505007,19.855713,20.109264,20.295783
2,,ARIMA,96.231022,12.0,9.0,16.0,16.0,18.0,0.0,6.0,...,13.0,20.0,15.0,14.0,14.504363,14.504363,14.504363,14.504363,14.504363,14.504363
3,,3MA,126.113845,12.0,9.0,16.0,16.0,18.0,0.0,6.0,...,13.0,20.0,15.0,14.0,32.464995,32.464995,30.320943,30.320943,26.837097,22.038769
4,,Linear_R,50.456223,12.0,9.0,16.0,16.0,18.0,0.0,6.0,...,13.0,20.0,15.0,14.0,15.742791,18.025722,17.340561,17.797090,18.558356,18.734792
5,,XG,58.548716,12.0,9.0,16.0,16.0,18.0,0.0,6.0,...,13.0,20.0,15.0,14.0,14.000104,16.537073,16.195595,16.371428,15.675302,15.672584
6,,SVR,47.826117,12.0,9.0,16.0,16.0,18.0,0.0,6.0,...,13.0,20.0,15.0,14.0,17.569120,19.015587,18.381045,18.919768,19.292457,19.311351
7,,RandomF,47.674726,12.0,9.0,16.0,16.0,18.0,0.0,6.0,...,13.0,20.0,15.0,14.0,15.500000,18.136594,18.368569,19.369406,20.628971,20.915621
1,600307,GARCH,1798.759585,5797.0,5017.0,7081.0,4250.0,2856.0,7075.0,1975.0,...,1103.0,388.0,103.0,167.0,3751.217365,3742.765904,3734.680053,3726.945118,3719.546909,3712.471727
2,,ARIMA,1062.341974,5797.0,5017.0,7081.0,4250.0,2856.0,7075.0,1975.0,...,1103.0,388.0,103.0,167.0,1470.384353,1470.384353,1470.384353,1470.384353,1470.384353,1470.384353
3,,3MA,1139.275369,5797.0,5017.0,7081.0,4250.0,2856.0,7075.0,1975.0,...,1103.0,388.0,103.0,167.0,6392.333333,6392.333333,6884.333333,6884.333333,6884.333333,6651.000000


In [ ]:
final_df.columns = list(final_df.columns[:3]) + list(actual_d.index.astype(str)) + list(x1.index.astype(str))
final_df_selected.columns = list(final_df_selected.columns[:3]) + list(actual_d.index.astype(str)) + list(x1.index.astype(str))

In [ ]:
combined_df

,0,1,2,3,4,5,6,7
1,845.000000,1471.000000,2967.738501,2581.000000,1681.178902,666.047974,2110.308161,1478.403067
2,1163.000000,1679.744123,2967.674572,2141.333333,-586.113722,845.000122,1975.833113,917.800878
3,1423.000000,1944.000000,2967.636393,2089.000000,1093.098559,1403.395142,1944.390535,1149.087597
4,923.000000,1044.000000,2967.613593,2042.666667,1199.922129,1053.737915,1573.248690,1265.020078
5,784.000000,725.000000,2967.599978,1932.914708,868.688112,1059.286499,1466.068627,1122.560000
6,739.000000,1289.000000,2967.599978,1698.248041,1164.446805,933.222595,1811.760656,978.093359
7,622.000000,1017.039647,2259.081001,1637.581374,1062.402778,914.130798,1572.644437,905.730000
8,782.000000,1091.367304,2259.016118,1465.333333,955.387425,979.994385,1465.632232,970.260000
9,1159.171252,1250.550343,2258.977389,1291.666667,693.147037,760.096558,1482.800841,753.050000
10,1027.000000,1096.122838,2258.954273,1291.666667,816.969278,717.180176,1425.292933,910.177951


In [ ]:
final_df

,idx,model,MAPE,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,...,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01
1,600153,GARCH,100.058283,12.0,9.0,16.0,16.0,18.0,0.0,6.0,...,13.0,20.0,15.0,14.0,18.254167,19.005672,19.505007,19.855713,20.109264,20.295783
2,,ARIMA,96.231022,12.0,9.0,16.0,16.0,18.0,0.0,6.0,...,13.0,20.0,15.0,14.0,14.504363,14.504363,14.504363,14.504363,14.504363,14.504363
3,,3MA,126.113845,12.0,9.0,16.0,16.0,18.0,0.0,6.0,...,13.0,20.0,15.0,14.0,32.464995,32.464995,30.320943,30.320943,26.837097,22.038769
4,,Linear_R,50.456223,12.0,9.0,16.0,16.0,18.0,0.0,6.0,...,13.0,20.0,15.0,14.0,15.742791,18.025722,17.340561,17.797090,18.558356,18.734792
5,,XG,58.548716,12.0,9.0,16.0,16.0,18.0,0.0,6.0,...,13.0,20.0,15.0,14.0,14.000104,16.537073,16.195595,16.371428,15.675302,15.672584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,,3MA,89.321374,845.0,1163.0,1423.0,923.0,784.0,739.0,622.0,...,1958.0,1027.0,973.0,1413.0,1637.581374,1465.333333,1291.666667,1291.666667,1291.666667,1169.666667
4,,Linear_R,49.484623,845.0,1163.0,1423.0,923.0,784.0,739.0,622.0,...,1958.0,1027.0,973.0,1413.0,1129.628861,1171.669794,1226.533765,1238.433551,1269.567090,1301.121159
5,,XG,26.345478,845.0,1163.0,1423.0,923.0,784.0,739.0,622.0,...,1958.0,1027.0,973.0,1413.0,1412.999146,1168.346313,1274.648438,1303.579468,1424.601318,1199.206909
6,,SVR,76.557042,845.0,1163.0,1423.0,923.0,784.0,739.0,622.0,...,1958.0,1027.0,973.0,1413.0,1285.627029,1286.267087,1286.690146,1286.957680,1286.961251,1286.962883


In [ ]:
final_df_selected

,idx,model,MAPE,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,...,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01
6,600153,SVR,47.826117,12.0,9.0,16.0,16.0,18.0,0.0,6.0,...,13.0,20.0,15.0,14.0,17.569120,19.015587,18.381045,18.919768,19.292457,19.311351
5,600307,XG,589.529604,5797.0,5017.0,7081.0,4250.0,2856.0,7075.0,1975.0,...,1103.0,388.0,103.0,167.0,166.999176,103.000832,2944.914062,166.999176,144.578857,6209.550781
5,600308,XG,27.889837,4693.0,9032.0,5403.0,14396.0,1864.0,317.0,46.0,...,13837.0,6867.0,5866.0,6184.0,6184.000977,7357.811523,7294.718750,7527.279785,7675.683105,7755.641113
5,600310,XG,90.362383,6.0,3.0,11.0,4.0,1.0,18.0,16.0,...,22.0,34.0,40.0,NaN,29.923056,25.768761,25.954893,29.920465,29.920465,29.920465
6,600311,SVR,25.454467,9801.0,6779.0,4646.0,1326.0,12898.0,21058.0,11054.0,...,9334.0,9249.0,9962.0,11511.0,9012.953436,9012.151389,9012.330795,9012.000613,9011.999548,9011.999824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,608640,XG,597.963232,8962.0,3659.0,1260.0,413.0,360.0,795.0,125.0,...,194.0,125.0,83.0,34.0,34.001778,41.791721,124.999207,74.642494,41.231598,68.663055
7,608923,RandomF,9.843762,2358.0,2181.0,2557.0,3492.0,1801.0,2778.0,2335.0,...,2887.0,2461.0,3324.0,3352.0,3028.200000,2456.710000,2745.557746,2748.161709,2461.714078,2509.551256
7,608925,RandomF,13.723722,1378.0,1871.0,1514.0,1791.0,1652.0,1665.0,1807.0,...,1873.0,1146.0,3178.0,2493.0,2256.986160,1859.745461,1771.480524,2199.992517,1708.753689,1754.050844
6,608926,SVR,13.504852,933.0,1095.0,1114.0,1107.0,931.0,944.0,1215.0,...,1028.0,798.0,1770.0,1111.0,1101.903084,1100.234102,1101.022304,1101.344797,1101.336130,1101.338826


In [ ]:
new_df

,609456,shifted_t1,shifted_t2,shifted_t3
date,,,,
2021-06-01,1203.000000,1940.489796,2700.000000,2272.000000
2021-07-01,2135.270304,1203.000000,1940.489796,2700.000000
2021-08-01,2624.335931,2135.270304,1203.000000,1940.489796
2021-09-01,2281.000000,2624.335931,2135.270304,1203.000000
2021-10-01,3402.000000,2281.000000,2624.335931,2135.270304
2021-11-01,2166.000000,3402.000000,2281.000000,2624.335931
2021-12-01,2083.000000,2166.000000,3402.000000,2281.000000
2022-01-01,2009.000000,2083.000000,2166.000000,3402.000000
2022-02-01,2175.000000,2009.000000,2083.000000,2166.000000


In [ ]:
mapes=final_df_selected.iloc[:,2]
quartile = mapes.values

In [ ]:
third_quartile = mapes.quantile(0.75)


In [ ]:
less_20 = (mapes < 20).sum()
bet_20_and_30 = ((mapes >= 20) & (mapes <= 30)).sum()
bet_30_and_50 = ((mapes >= 30) & (mapes <= 50)).sum()
greater_50 = (mapes > 50).sum()

In [ ]:
report=pd.DataFrame({'Accuracy':['less than 20','between 20 and 30','between 30 and 50','Greater than 50','Total']})
report['Count']=[less_20,bet_20_and_30,bet_30_and_50,greater_50,len(mapes)]
report['Percent']=[less_20/len(mapes),bet_20_and_30/len(mapes),bet_30_and_50/len(mapes),greater_50/len(mapes),'']
report[""]=""
report['3rd Quartile MAPE'] = ""
report.loc[0, '3rd Quartile MAPE'] = third_quartile
print(report)

            Accuracy  Count   Percent   3rd Quartile MAPE
0       less than 20    283  0.478849           39.627003
1  between 20 and 30    109  0.184433                    
2  between 30 and 50     75  0.126904                    
3    Greater than 50    124  0.209814                    
4              Total    591                              


In [ ]:
writer = pd.ExcelWriter('/media/thilan/Coding/Hemas/output_newex_A.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, sheet_name='Forecast')
final_df_selected.to_excel(writer, sheet_name='Final_Forecast')
report.to_excel(writer, sheet_name='Accuracy_Report')


worksheet = writer.sheets['Accuracy_Report']
start_row = worksheet.dim_rowmax + 3
df_removed.to_excel(writer, sheet_name='Accuracy_Report', startrow=start_row, index=False)


workbook = writer.book
worksheet = writer.sheets['Forecast']
highlight_format = workbook.add_format({'bg_color': 'yellow'})
format_red = workbook.add_format({'bg_color': '#FF0000'})

for col_num in range(16, 22):
    worksheet.set_column(col_num, col_num, None, highlight_format)


worksheet = writer.sheets['Final_Forecast']
highlight_format = workbook.add_format({'bg_color': 'yellow'})

worksheet.conditional_format('D2:D300', {'type': 'cell', 'criteria': '>', 'value': 50, 'format': format_red})

for col_num in range(16, 22):
    worksheet.set_column(col_num, col_num, None, highlight_format)
    worksheet.set_column(col_num, col_num, None)


writer.save()

print('Time elapsed: ',time.time() - start_time)
convert_seconds_to_hours_minutes(time.time() - start_time)

Time elapsed:  16668.358137369156
Time elapsed -  4.0 hours and  37.0 minutes


In [ ]:
final_df_selected['MAPE']

6     47.826117
5    589.529604
5     27.889837
5     90.362383
6     25.454467
        ...    
5    597.963232
7      9.843762
7     13.723722
6     13.504852
5     26.345478
Name: MAPE, Length: 591, dtype: float64